# Advanced Algorithmic problem in Lists

### Stock Buy and Sell problem

The cost of a stock on each day is given in an array. Find the maximum profit that you can make by buying and selling on those days. If the given array of prices is sorted in decreasing order, then profit cannot be earned at all.

# Stock Buy and Sell — Revision Notes

We are given an array `price[]` where `price[i]` is the price of a stock on day `i`.
We want to maximize profit under the rule:

* We may buy and sell multiple times, but must **sell before buying again**.

---

## Example

```text
Input:  [100, 180, 260, 310, 40, 535, 695]
Output: 865

Explanation:
- Buy on day 0, sell on day 3 → Profit = 310 - 100 = 210
- Buy on day 4, sell on day 6 → Profit = 695 - 40 = 655
Total Profit = 865
```

---

## Approach 1 — Naïve Recursion (Exponential)

### Key idea

Pick a transaction `(i, j)` with `price[j] > price[i]`. If you fix that transaction, the array splits into two independent parts:

* left: indices `start` .. `i-1`
* right: indices `j+1` .. `end`

Total profit when choosing `(i,j)`:

```
Profit(i,j) = (price[j] - price[i])
            + maxProfit(start, i-1)
            + maxProfit(j+1, end)
```

Try all valid `(i,j)` and return the maximum.

### Pseudocode

```python
def maxProfit(prices, l, r):
    # returns max profit in prices[l..r]
    if l >= r:
        return 0
    best = 0
    for i in range(l, r):
        for j in range(i+1, r+1):
            if prices[j] > prices[i]:
                profit = prices[j] - prices[i]
                profit += maxProfit(prices, l, i-1)
                profit += maxProfit(prices, j+1, r)
                best = max(best, profit)
    return best

# final answer: maxProfit(prices, 0, n-1)
```

**Complexity:** exponential in general (overlapping subproblems). With memoization over `(l,r)` pairs there are `O(n^2)` subproblems, but each does `O(n^2)` work inside → still expensive.

---

## Worked example (Naïve recursion)

Array: `[1, 5, 3, 8, 12]`

* Consider pair `(0,1)` → buy at `1`, sell at `5`

  * direct profit = `5 - 1 = 4`
  * left = `maxProfit(0, -1)` = 0
  * right = `maxProfit(2, 4)` on `[3, 8, 12]` → best there = 9 (from `(3,12)`)
  * total = `4 + 0 + 9 = 13`

Trying all pairs yields maximum `13` (for example: buy\@1 sell\@5, then buy\@3 sell\@12).

---

## Recursive tree (compact) for `[1,5,3,8,12]`

Call: `maxProfit(0,4)` → `[1,5,3,8,12]`

* (0,1) → total = 13
* (0,2) → total = 6
* (0,3) → total = 7
* (0,4) → total = 11
* (1,3) → total = 3
* (1,4) → total = 7
* (2,3) → total = 9
* (2,4) → total = 13
* (3,4) → total = 8

**Best = 13**

(Internal calculations show `maxProfit(2,4) = 9` because among `(3,8)=5`, `(3,12)=9`, `(8,12)=4`, the max is 9.)

---

## Approach 2 — Local Minima & Local Maxima (O(N))

Find intervals `[buy, sell]` where the price goes up from a local minima to the next local maxima. Repeat until the end.

### Algorithm (prints buy/sell days)

```python
def stockBuySell(price):
    n = len(price)
    if n < 2:
        return
    i = 0
    while i < n-1:
        # find local minima (buy)
        while i < n-1 and price[i+1] <= price[i]:
            i += 1
        if i == n-1:
            break
        buy = i
        i += 1
        # find local maxima (sell)
        while i < n and price[i] >= price[i-1]:
            i += 1
        sell = i-1
        print(f"Buy on day: {buy}, Sell on day: {sell}")
```

**Complexity:** O(N) time, O(1) extra space.

---

## Approach 3 — Valley–Peak (Greedy O(N))

Observation: adding all positive consecutive differences equals taking all valley→peak trades.

```python
def max_profit(prices):
    profit = 0
    for i in range(1, len(prices)):
        if prices[i] > prices[i-1]:
            profit += prices[i] - prices[i-1]
    return profit
```

**Complexity:** O(N) time, O(1) space.

---

## Special case — Only one transaction allowed (LeetCode 121)

If you can make at most one buy→sell, track `min_price` and `max_profit`:

```python
def maxProfitOneTransaction(prices):
    min_price = float('inf')
    max_profit = 0
    for p in prices:
        min_price = min(min_price, p)
        max_profit = max(max_profit, p - min_price)
    return max_profit
```

---

## Comparison table

| Approach               | Time Complexity | Space                | When to use                                  |
| ---------------------- | --------------- | -------------------- | -------------------------------------------- |
| Naïve recursion        | Exponential     | O(n) recursion depth | Teaching / correctness proofs                |
| Local Min / Local Max  | O(N)            | O(1)                 | When you need actual buy/sell days           |
| Valley–Peak (Greedy)   | O(N)            | O(1)                 | Fast total profit for unlimited transactions |
| One-transaction method | O(N)            | O(1)                 | If only one buy-sell allowed (LeetCode 121)  |

---

## Quick summary

* Naïve recursion directly follows the split idea but is inefficient.
* The valley–peak (or summing positive diffs) is the simplest optimal solution for multiple transactions.
* If the question restricts to a single transaction, use the min-tracking method.


# Stock Buy & Sell Strategies

## 1. Valley–Peak (Greedy / O(n))

**Idea:**  
Every time `price[i] > price[i-1]`, we can "buy at day (i-1), sell at day i" and add the profit.  
This is equivalent to summing all positive differences.

**Example:** `[1, 5, 3, 8, 12]`

| Day | Price | Δ = price[i] - price[i-1] | Profit Added | Total Profit |
|-----|-------|---------------------------|--------------|--------------|
| 0   | 1     | —                         | 0            | 0            |
| 1   | 5     | 5 - 1 = 4                 | +4           | 4            |
| 2   | 3     | 3 - 5 = -2 (skip)         | 0            | 4            |
| 3   | 8     | 8 - 3 = 5                 | +5           | 9            |
| 4   | 12    | 12 - 8 = 4                | +4           | 13           |

✅ **Final profit = 13**

---

## 2. Local Minima & Maxima Method

**Idea:**  
1. Find **local minima** → buy point.  
2. Find **next local maxima** → sell point.  
3. Repeat until the end of the array.

**Example:** `[1, 5, 3, 8, 12]`

- `price[0] = 1` → local minima → buy
- Climb to 5 → local maxima at index 1 → sell  
  Transaction: Buy 1, Sell 5 → profit 4
- After 5, price drops to 3 → local minima → buy
- Climb to 12 → local maxima at index 4 → sell  
  Transaction: Buy 3, Sell 12 → profit 9

✅ **Total profit = 4 + 9 = 13**

---

## 3. Comparison with Naïve Recursive

- Naïve recursion explores all pairs of buy-sell days → eventually finds `(1,5)` and `(3,12)` giving 13.
- **Greedy (sum of positive differences)** → gives 13 in **O(n)**.
- **Local minima/maxima method** → also gives 13 in **O(n)**.




# Traprain Water

# Trapping Rain Water Problem Notes

## Problem Statement
Given an array `height[]` of non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it can trap after raining.

**Example 1:**
- Input: `height = [2,0,2]`
- Output: `2`
- Explanation: Water trapped between the two bars of height 2.

**Example 2:**
- Input: `height = [3,0,2,0,4]`
- Output: `7`
- Explanation: Water trapped = 3 + 1 + 3 = 7

---

## Intuition
- Water is trapped at a bar if there are taller bars on **both sides**.  
- Water trapped at index `i` = `min(max_left[i], max_right[i]) - height[i]`.  
- Total water = sum of water trapped at all indices.

---

## Approach 1: Brute Force

**Idea:** For each bar, find the tallest bar on its left and right. Calculate water trapped at this bar.

**Steps:**
1. Traverse each element `i` from `1` to `n-2`.
2. Find `max_left` for index `i` (max height from 0 to i).
3. Find `max_right` for index `i` (max height from i to n-1).
4. Water trapped = `min(max_left, max_right) - height[i]`.
5. Add to result.

**Code (Indented Block):**
```python
    def maxWater(arr, n):
        res = 0
        for i in range(1, n-1):
            left = max(arr[:i+1])
            right = max(arr[i:])
            res += min(left, right) - arr[i]
        return res

    arr = [3,0,2,0,4]
    print(maxWater(arr, len(arr)))  # Output: 7
```
**Dry Run Example:**  
`arr = [3,0,2,0,4]`

| i | left_max | right_max | water | total |
|---|----------|-----------|-------|-------|
| 1 | 3        | 4         | 3     | 3     |
| 2 | 3        | 4         | 1     | 4     |
| 3 | 3        | 4         | 3     | 7     |

**Complexity:**  
- Time: O(N²)  
- Space: O(1)

---

## Approach 2: Precomputation (DP)

**Idea:** Precompute left_max and right_max for all bars to reduce repeated calculations.

**Steps:**
1. Create `left[i]` = max height from start to i.  
2. Create `right[i]` = max height from i to end.  
3. Water trapped at index i = `min(left[i], right[i]) - height[i]`.  
4. Sum up water for all bars.
```python
    def findWater(arr, n):
        left = [0]*n
        right = [0]*n
        water = 0

        left[0] = arr[0]
        for i in range(1, n):
            left[i] = max(left[i-1], arr[i])

        right[n-1] = arr[n-1]
        for i in range(n-2, -1, -1):
            right[i] = max(right[i+1], arr[i])

        for i in range(n):
            water += min(left[i], right[i]) - arr[i]

        return water

    arr = [3,0,2,0,4]
    print(findWater(arr, len(arr)))  # Output: 7
```
**Dry Run Example:**  

`arr   = [3,0,2,0,4]`  
`left  = [3,3,3,3,4]`  
`right = [4,4,4,4,4]`  

| i | min(left[i], right[i]) | height[i] | water | total |
|---|------------------------|-----------|-------|-------|
| 0 | 3                      | 3         | 0     | 0     |
| 1 | 3                      | 0         | 3     | 3     |
| 2 | 3                      | 2         | 1     | 4     |
| 3 | 3                      | 0         | 3     | 7     |
| 4 | 4                      | 4         | 0     | 7     |

**Complexity:**  
- Time: O(N)  
- Space: O(N)

---

## Approach 3: Two-Pointer (Optimal)

**Idea:** Use two pointers (`left` and `right`) and maintain `lmax` and `rmax`. Water trapped depends on the smaller side.

**Steps:**
1. Initialize `left=0`, `right=n-1`, `lmax=0`, `rmax=0`, `res=0`.  
2. While `left <= right`:
   - If `height[left] < height[right]`:
     - `lmax = max(lmax, height[left])`
     - Water trapped = `lmax - height[left]`
     - `left += 1`
   - Else:
     - `rmax = max(rmax, height[right])`
     - Water trapped = `rmax - height[right]`
     - `right -= 1`
3. Return `res`.


```python
    class Solution:
        def trap(self, height):
            left, right = 0, len(height)-1
            lmax = rmax = res = 0

            while left <= right:
                if height[left] < height[right]:
                    lmax = max(lmax, height[left])
                    res += lmax - height[left]
                    left += 1
                else:
                    rmax = max(rmax, height[right])
                    res += rmax - height[right]
                    right -= 1
            return res

    height = [3,0,2,0,4]
    sol = Solution()
    print(sol.trap(height))  # Output: 7
```
**Dry Run Example:**  

`height = [3,0,2,0,4]`  

- Step 1: left=0, right=4 → height[left]=3 < height[right]=4 → lmax=3 → res+=0 → left=1  
- Step 2: left=1, right=4 → height[left]=0 < 4 → lmax=3 → res+=3 → left=2  
- Step 3: left=2, right=4 → height[left]=2 < 4 → lmax=3 → res+=1 → left=3  
- Step 4: left=3, right=4 → height[left]=0 < 4 → lmax=3 → res+=3 → left=4  
- Step 5: left=4, right=4 → height[left]=4 !< 4 → rmax=4 → res+=0 → right=3  

**Total water trapped = 7**

**Complexity:**  
- Time: O(N)  
- Space: O(1)

---

## Summary Table

| Approach              | Time Complexity | Space Complexity | Notes                    |
|-----------------------|----------------|----------------|--------------------------|
| Brute Force           | O(N²)          | O(1)           | Simple, inefficient      |
| Precomputation (DP)   | O(N)           | O(N)           | Uses extra arrays        |
| Two-Pointer (Optimal) | O(N)           | O(1)           | Most efficient approach  |


# Maximum Subarray Sum (Any Size)

## Problem Description

Given an array of integers, find the maximum sum of a contiguous subarray (any size).

* **Input:** array `arr` of integers
* **Output:** Maximum sum of any contiguous subarray

---

## Examples

**Example:**

```python
arr = [-2, -3, 4, -1, -2, 1, 5, -3]
```

**Output:**

```
7
```

(Maximum sum from subarray `[4, -1, -2, 1, 5]`)

---

## Naive Approach (O(n²))

```python
def max_subarray_naive(arr):
    n = len(arr)
    max_sum = float('-inf')
    for i in range(n):
        curr_sum = 0
        for j in range(i, n):
            curr_sum += arr[j]
            max_sum = max(max_sum, curr_sum)
    return max_sum

# Example usage
arr = [-2, -3, 4, -1, -2, 1, 5, -3]
print(max_subarray_naive(arr))  # Output: 7
```

* **Time Complexity:** O(n²)
* **Space Complexity:** O(1)

---

## Optimized Approach (Kadane’s Algorithm, O(n))

```python
def kadane(arr):
    max_so_far = max_ending_here = arr[0]
    for i in range(1, len(arr)):
        max_ending_here = max(arr[i], max_ending_here + arr[i])
        max_so_far = max(max_so_far, max_ending_here)
    return max_so_far

# Example usage
arr = [-2, -3, 4, -1, -2, 1, 5, -3]
print(kadane(arr))  # Output: 7
```

* **Time Complexity:** O(n)
* **Space Complexity:** O(1)

---

## Explanation

* **Naive approach:** Check all possible subarrays and track maximum sum. Simple but inefficient for large arrays.
* **Kadane’s algorithm:** Maintain `max_ending_here` for current subarray sum and `max_so_far` for maximum found. Efficient and fast.

---

## References

* [GeeksforGeeks: Kadane's Algorithm](https://www.geeksforgeeks.org/largest-sum-contiguous-subarray/)


# Maximum Subarray Sum

## Problem Description

Given an array of integers and a number `k`, find the maximum sum of a contiguous subarray of size `k`.

* **Input:** array `arr` of integers, integer `k`
* **Output:** Maximum sum of any contiguous subarray of size `k`.
* If the array size is less than `k`, return "Invalid".

---

## Examples

**Example 1:**

```python
arr = [100, 200, 300, 400]
k = 2
```

**Output:**

```
700
```

**Example 2:**

```python
arr = [1, 4, 2, 10, 23, 3, 1, 0, 20]
k = 4
```

**Output:**

```
39
```

(Maximum sum from subarray `[4, 2, 10, 23]`)

**Example 3:**

```python
arr = [2, 3]
k = 3
```

**Output:**

```
Invalid
```

(No subarray of size 3 exists)

---

## Efficient Solution (Sliding Window)

```python
def maxSum(arr, k):
    n = len(arr)
    if n < k:
        return "Invalid"

    curr_sum = max_sum = sum(arr[:k])
    for i in range(k, n):
        curr_sum += arr[i] - arr[i - k]
        max_sum = max(max_sum, curr_sum)

    return max_sum

# Example usage
arr = [1, 4, 2, 10, 23, 3, 1, 0, 20]
k = 4
print(maxSum(arr, k))  # Output: 39
```

* **Time Complexity:** O(n)
* **Space Complexity:** O(1)

---

## Maximum Sum Subarray (Any Size) — Kadane's Algorithm

```python
def kadane(arr):
    max_so_far = max_ending_here = arr[0]
    for i in range(1, len(arr)):
        max_ending_here = max(arr[i], max_ending_here + arr[i])
        max_so_far = max(max_so_far, max_ending_here)
    return max_so_far

# Example usage
arr = [-2, -3, 4, -1, -2, 1, 5, -3]
print(kadane(arr))  # Output: 7
```

* **Time Complexity:** O(n)
* **Space Complexity:** O(1)

---

## References

* [GeeksforGeeks: Maximum Sum Subarray of Size k](https://www.geeksforgeeks.org/maximum-of-all-subarrays-of-size-k/)
* [GeeksforGeeks: Kadane's Algorithm](https://www.geeksforgeeks.org/largest-sum-contiguous-subarray/)
